In [3]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np
!pip install -U imbalanced-learn
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.datasets import load_digits
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

## 데이터 처리

In [4]:
data=pd.read_csv(r"WSDM_train_full_features.csv")
data=data.sample(frac=0.05, random_state=0)
data.shape

(368871, 30)

In [5]:
data.head()

,target,user_idx,song_idx,ssn_idx,sst_idx,st_idx,song_length,genr_idx,art_idx,com_idx,...,lyricists_count,composer_count,is_featured,artist_count,artist_composer,artist_composer_lyricist,song_lang_boolean,smaller_song,count_song_played,count_artist_played
4536199,1,9481,8945,1,1,2,233116,0,146,2,...,0,1,0,0,0,0,0,1,202,2771
2588100,0,15666,20,8,3,0,189846,4,3339,8365,...,0,1,0,0,0,0,0,1,3496,6250
1650329,1,9919,7269,5,1,4,287137,0,166,12577,...,0,1,0,0,0,0,0,0,133,33035
2732406,0,19109,118170,5,3,4,192783,25,278,188,...,0,1,0,0,1,0,0,1,7,540
613088,1,818,6790,1,1,1,221309,1,11967,2,...,0,1,0,0,0,0,0,1,11,26


In [6]:
data_x=data.iloc[:,1:];    data_y=data.iloc[:,0]

In [7]:
len_0=len(data.query("target==0"))
len_1=len(data.query("target==1"))
imbalacned_list_p={0.001:999,0.01:99,0.1:9}

def making_imbalanced(x):
    i=imbalacned_list_p[x]
    len_2=round((len_0/i*(i+1))-len_0)
    data_target1=data.query("target==1").sample(n=len_2,random_state=42)
    data_final=pd.concat([data.query("target==0"),data_target1])
    print(len(data_target1)/len(data_final))
    return data_final

In [8]:
data_1=making_imbalanced(0.1)
data_x1=data_1.iloc[:,1:];    data_y1=data_1.iloc[:,0]
data_01=making_imbalanced(0.01)
data_x01=data_01.iloc[:,1:];    data_y01=data_01.iloc[:,0]
data_001=making_imbalanced(0.001)
data_x001=data_001.iloc[:,1:];    data_y001=data_001.iloc[:,0]


0.10000196631698996
0.009998269634243937
0.0009985485578339682


In [9]:
score_list=[]
def imbalanced_process(x,x_1,y,y_1,z):
    if z!=0:
        X1, y1 = z.fit_resample(x, y)
    else: X1,y1=x,y
    RF=RandomForestClassifier(random_state=0)
    RF.fit(X1, y1)
    y1_pred=RF.predict(x_1)
    y2_pred=RF.predict_proba(x_1)[:,1]
    print(f'accuracy_score : {round(accuracy_score(y_1, y1_pred),5)}, roc_auc_score : {round(roc_auc_score(y_1, y2_pred),5)}, f1_score : {round(f1_score(y_1, y1_pred),5)}, precision_score : {round(precision_score(y_1, y1_pred),5)}, recall_score : {round(recall_score(y_1, y1_pred),5)}')
    score_list.append([round(accuracy_score(y_1, y1_pred),5),round(roc_auc_score(y_1, y2_pred),5),round(f1_score(y_1, y1_pred),5),round(precision_score(y_1, y1_pred),5),round(recall_score(y_1, y1_pred),5)])

## 999:1 imbalanced

In [10]:
# 학습/평가 데이터 분할
X001_train, X001_test, y001_train, y001_test = train_test_split(data_x001, data_y001, random_state=0)
X001_train.shape, X001_test.shape

((137449, 29), (45817, 29))

### original

In [97]:
from sklearn.ensemble import RandomForestClassifier
# 원래 데이터를 사용했을 때의 모델성능
imbalanced_process(X001_train, X001_test, y001_train, y001_test,0)

accuracy_score : 0.99895, roc_auc_score : 0.52608, f1_score : 0.0, precision_score : 0.0, recall_score : 0.0


/Users/jody/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jody/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Under_Sampling

- Ramdom Under-Sampler

In [98]:
from imblearn.under_sampling import *
from sklearn.ensemble import RandomForestClassifier
imbalanced_process(X001_train, X001_test, y001_train, y001_test, RandomUnderSampler(random_state=0))

accuracy_score : 0.59783, roc_auc_score : 0.55891, f1_score : 0.00249, precision_score : 0.00125, recall_score : 0.47917


- NearMiss-1

In [99]:
from imblearn.under_sampling import *
from sklearn.ensemble import RandomForestClassifier
imbalanced_process(X001_train, X001_test, y001_train, y001_test, NearMiss(version=1,n_jobs=-1))

accuracy_score : 0.01721, roc_auc_score : 0.53044, f1_score : 0.00213, precision_score : 0.00107, recall_score : 1.0


- NearMiss-2

In [100]:
from imblearn.under_sampling import *
from sklearn.ensemble import RandomForestClassifier
imbalanced_process(X001_train, X001_test, y001_train, y001_test, NearMiss(version=2,n_jobs=-1))

accuracy_score : 0.01926, roc_auc_score : 0.5595, f1_score : 0.00213, precision_score : 0.00107, recall_score : 1.0


- NearMiss-3

In [101]:
from imblearn.under_sampling import *
from sklearn.ensemble import RandomForestClassifier
imbalanced_process(X001_train, X001_test, y001_train, y001_test, NearMiss(version=3,n_jobs=-1))

accuracy_score : 0.45236, roc_auc_score : 0.56272, f1_score : 0.00239, precision_score : 0.0012, recall_score : 0.625


- Condensed Nearest Neighbour

In [136]:
from imblearn.under_sampling import *
from sklearn.ensemble import RandomForestClassifier
imbalanced_process(X001_train, X001_test, y001_train, y001_test, CondensedNearestNeighbour(random_state=0))

accuracy_score : 0.99871, roc_auc_score : 0.53652, f1_score : 0.0, precision_score : 0.0, recall_score : 0.0


- TomekLinks

In [103]:
from imblearn.under_sampling import *
from sklearn.ensemble import RandomForestClassifier
imbalanced_process(X001_train, X001_test, y001_train, y001_test, TomekLinks(sampling_strategy='all'))

accuracy_score : 0.99895, roc_auc_score : 0.5282, f1_score : 0.0, precision_score : 0.0, recall_score : 0.0


/Users/jody/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jody/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


- Edited Nearest Neighbours

In [104]:
from imblearn.under_sampling import *
from sklearn.ensemble import RandomForestClassifier
imbalanced_process(X001_train, X001_test, y001_train, y001_test, EditedNearestNeighbours(kind_sel="all", n_neighbors=5))

accuracy_score : 0.99893, roc_auc_score : 0.51332, f1_score : 0.0, precision_score : 0.0, recall_score : 0.0


- RepeatedEditedNearestNeighbours

In [115]:
from imblearn.under_sampling import *
from sklearn.ensemble import RandomForestClassifier
imbalanced_process(X001_train, X001_test, y001_train, y001_test, RepeatedEditedNearestNeighbours(kind_sel="all", n_neighbors=5))

accuracy_score : 0.99893, roc_auc_score : 0.51446, f1_score : 0.0, precision_score : 0.0, recall_score : 0.0


### Over_Sampling

- RandomOverSampler

In [107]:
from imblearn.over_sampling import *
from sklearn.ensemble import RandomForestClassifier
imbalanced_process(X001_train, X001_test, y001_train, y001_test, RandomOverSampler(random_state=0))

accuracy_score : 0.99891, roc_auc_score : 0.50943, f1_score : 0.0, precision_score : 0.0, recall_score : 0.0


- SMOTE

In [108]:
from imblearn.over_sampling import *
from sklearn.ensemble import RandomForestClassifier
imbalanced_process(X001_train, X001_test, y001_train, y001_test,SMOTE(random_state=0,k_neighbors=5))

accuracy_score : 0.99882, roc_auc_score : 0.51792, f1_score : 0.0, precision_score : 0.0, recall_score : 0.0


- BorderlineSMOTE-1

In [109]:
from imblearn.over_sampling import *
from sklearn.ensemble import RandomForestClassifier
imbalanced_process(X001_train, X001_test, y001_train, y001_test,BorderlineSMOTE(kind = 'borderline-1',random_state=0))

accuracy_score : 0.99893, roc_auc_score : 0.49569, f1_score : 0.0, precision_score : 0.0, recall_score : 0.0


- BorderlineSMOTE-2

In [110]:
from imblearn.over_sampling import *
from sklearn.ensemble import RandomForestClassifier
imbalanced_process(X001_train, X001_test, y001_train, y001_test,BorderlineSMOTE(kind = 'borderline-2',random_state=0))

accuracy_score : 0.99893, roc_auc_score : 0.50487, f1_score : 0.0, precision_score : 0.0, recall_score : 0.0


### Combine

- SMOTETomek

In [111]:
from imblearn.combine import *
from sklearn.ensemble import RandomForestClassifier
imbalanced_process(X001_train, X001_test, y001_train, y001_test,SMOTETomek(random_state=0, sampling_strategy='all'))

accuracy_score : 0.99882, roc_auc_score : 0.51792, f1_score : 0.0, precision_score : 0.0, recall_score : 0.0


- SMOTE+ENN

In [112]:
from imblearn.combine import *
from sklearn.ensemble import RandomForestClassifier
imbalanced_process(X001_train, X001_test, y001_train, y001_test,SMOTEENN(random_state=0))

accuracy_score : 0.99882, roc_auc_score : 0.52802, f1_score : 0.0, precision_score : 0.0, recall_score : 0.0


### Ensemble

- BalanceCascade

In [113]:
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
brf.fit(X001_train, y001_train) 
y1_pred = brf.predict(X001_test)
y2_pred = brf.predict_proba(X001_test)[:,1]
print(f'accuracy_score : {round(accuracy_score(y001_test, y1_pred),5)}, roc_auc_score : {round(roc_auc_score(y001_test, y2_pred),5)}, f1_score : {round(f1_score(y001_test, y1_pred),5)}, precision_score : {round(precision_score(y001_test, y1_pred),5)}, recall_score : {round(recall_score(y001_test, y1_pred),5)}')

accuracy_score : 0.62356, roc_auc_score : 0.58664, f1_score : 0.00255, precision_score : 0.00128, recall_score : 0.45833


- EasyEnsemble

In [116]:
from imblearn.ensemble import BalancedRandomForestClassifier
eec=EasyEnsembleClassifier(random_state=0)
eec.fit(X001_train, y001_train) 
y1_pred = eec.predict(X001_test)
y2_pred = eec.predict_proba(X001_test)[:,1]
print(f'accuracy_score : {round(accuracy_score(y001_test, y1_pred),5)}, roc_auc_score : {round(roc_auc_score(y001_test, y2_pred),5)}, f1_score : {round(f1_score(y001_test, y1_pred),5)}, precision_score : {round(precision_score(y001_test, y1_pred),5)}, recall_score : {round(recall_score(y001_test, y1_pred),5)}')

accuracy_score : 0.57082, roc_auc_score : 0.64453, f1_score : 0.00315, precision_score : 0.00158, recall_score : 0.64583
